# **PHP 2650: Statistical Learning Final Project**

*Run all cells except training for testing purposes*

In [1]:
# Uncomment to download tensorflow
# pip install tensorflow

## **Sepsis Patient Simulation**

In [2]:
import numpy as np             # Lin. Alg. Purposes
import random                  # Generating Random Numbers
import tensorflow as tf        # Deep Learning Model
from collections import deque  # Deque is a list with a maximum size (used for DQN-Memory)


class SepsisSimulation:
    def __init__(self):
        self.start_diagnosis = random.randint(1, 4) # 0 for Stable, 1 for SIRS, 2 for Sepsis, 3 for Severe Sepsis, and 4 for Septic Shock
        self.patient = self.generate_patient()      # Set generates initial patient
        self.time_elapsed = 0
        self.is_alive = True

    def generate_patient(self):
        # Generate a random patient with different vital signs
        age = random.randint(18, 90)
        sex = random.choice(["male", "female"])
        is_infected = False
        temperature = round(random.uniform(37.5, 38.5), 1)
        heart_rate = random.randint(60, 120)
        respiratory_rate = random.randint(12, 16)
        systolic_bp = random.randint(90, 120)
        diastolic_bp = random.randint(60, 80)
        lactate = random.uniform(0.5, 1.5)
        oxygen_saturation = random.randint(95, 100)

        # Alter Diagnostics based on True Diagnosis
        if self.start_diagnosis == 2:
            # Sepsis
            is_infected = True
            temperature += random.uniform(0, 1.0)
            heart_rate += random.randint(5, 20)
            respiratory_rate += random.randint(1, 3)
            lactate += random.uniform(0.5, 1.0)
            systolic_bp -= random.randint(5, 10)
            diastolic_bp -= random.randint(0, 5)
            oxygen_saturation -= random.randint(0, 5)
            
        elif self.start_diagnosis == 3:
            # Severe Sepsis
            is_infected = True
            temperature += random.uniform(.5, 1.0)
            heart_rate += random.randint(10, 20)
            respiratory_rate += random.randint(2, 4)
            lactate += random.uniform(2.0, 3.0)
            systolic_bp -= random.randint(5, 15)
            diastolic_bp -= random.randint(0, 7)
            oxygen_saturation -= random.randint(1, 10)

        elif self.start_diagnosis == 4:
            # Septic Shock
            is_infected = True
            temperature += random.uniform(1.0, 3.0)
            heart_rate += random.randint(20, 40)
            respiratory_rate += random.randint(3, 5)
            systolic_bp -= random.randint(10, 20)
            diastolic_bp -= random.randint(5, 10)
            lactate += random.uniform(3.0, 5.0)
            oxygen_saturation -= random.randint(5, 15)

        # Alter Diagnostics based on Age
        if age > 60:
            systolic_bp += random.randint(0, 20)
            diastolic_bp += random.randint(0, 10)
            respiratory_rate += random.randint(0, 2)
            lactate += random.uniform(0.0, 1.0)

        # Returns a Dictionary with Patient Information
        patient = {
            "age": age,
            "sex": sex,
            "temperature": temperature,
            "heart_rate": heart_rate,
            "respiratory_rate": respiratory_rate,
            "systolic_bp": systolic_bp,
            "diastolic_bp": diastolic_bp,
            "lactate": lactate,
            "oxygen_saturation": oxygen_saturation,
            "is_infected": is_infected, 
            "diagnosis": self.start_diagnosis 
        }

        return patient
    
    def new_patient(self, start_diagnosis=None):
        # Simulate a New Patient
        self.start_diagnosis = start_diagnosis or random.randint(1, 4)
        self.patient = self.generate_patient()
        self.time_elapsed = 0
        self.is_alive = True
    
    def step(self, action):
        ### Update Vitals Based on Action Chosen ###

        # Antipyretic (acetaminophen or ibuprofen)
        if action == 0 and self.patient["temperature"] > 37.5:
            self.patient["temperature"] = max(37.5, self.patient["temperature"] - random.uniform(1, 2))

        # Antibiotics
        if action == 1:
            self.patient["is_infected"] = False

        # Fluids
        elif action == 2:
            if self.patient["diagnosis"] == 4:
                self.patient["systolic_bp"] += random.randint(5, 20)
                self.patient["diastolic_bp"] += random.randint(5, 15)
            else:
                self.patient["systolic_bp"] += random.randint(3, 7)
                self.patient["diastolic_bp"] += random.randint(1, 5)
            self.patient["respiratory_rate"] = max(12, self.patient["respiratory_rate"] - random.randint(2, 4))
            self.patient["lactate"] = max(0, self.patient["lactate"] - random.uniform(1, 2))
            self.patient["temperature"] = max(37.5, self.patient["temperature"] - random.uniform(0.1, 0.3))

        # Vasopressors
        elif action == 3:
            if self.patient["diagnosis"] == 4:
                self.patient["systolic_bp"] += random.randint(10, 20)
                self.patient["diastolic_bp"] += random.randint(5, 15)
            else:
                self.patient["systolic_bp"] += random.randint(5, 10)
                self.patient["diastolic_bp"] += random.randint(3, 7)
            self.patient["lactate"] = max(0, self.patient["lactate"] - random.uniform(.5, 1.5))
            self.patient["oxygen_saturation"] = min(100, self.patient["oxygen_saturation"] + random.randint(1, 5))

        # Oxygen Therapy
        elif action == 4:
            self.patient["respiratory_rate"] = max(12, self.patient["respiratory_rate"] - random.randint(2, 4))
            self.patient["oxygen_saturation"] = min(100, self.patient["oxygen_saturation"] + random.randint(10, 15))


        ### Update Vitals and information for Round ###

        if self.patient["diagnosis"] == 1: # SIRS
            # Random Chance to Develop an infection
            self.patient["is_infected"] = random.choices([True, False], [.25, .75])[0]

            # Update Diagnosis 
            if self.patient["is_infected"]:
                self.patient["diagnosis"] = 2

            # Check if stable
            elif self.patient["systolic_bp"] >= 90 and self.patient["oxygen_saturation"] >= 95 and self.patient["respiratory_rate"] <= 16 \
                and self.patient["lactate"] < 4 and (36 < self.patient["temperature"] < 38):
                self.patient["diagnosis"] = 0

        elif self.patient["diagnosis"] == 2: # Sepsis
            # Update Vitals
            self.patient["systolic_bp"] -= random.randint(0, 5)
            self.patient["diastolic_bp"] -= random.randint(0, 5)
            self.patient["lactate"] += random.uniform(0, 1)
            self.patient["temperature"] += random.uniform(0, .1)
            self.patient["respiratory_rate"] += random.randint(0, 2)

            # Update Diagnosis
            if not self.patient["is_infected"] and self.patient["oxygen_saturation"] >= 95:
                self.patient["diagnosis"] = 1

            elif self.patient["lactate"] >= 4 or self.patient["systolic_bp"] < 90:
                self.patient["diagnosis"] = 3

        elif self.patient["diagnosis"] == 3: # Severe Sepsis
            # Update Vitals
            self.patient["systolic_bp"] -= random.randint(0, 5)
            self.patient["diastolic_bp"] -= random.randint(0, 5)
            self.patient["lactate"] += random.uniform(0, 1)
            self.patient["temperature"] += random.uniform(0, .2)
            self.patient["respiratory_rate"] += random.randint(0, 3)
            self.patient["oxygen_saturation"] -= random.randint(1, 3)

            # Update Diagnosis
            if self.patient["lactate"] < 4 and self.patient["systolic_bp"] >= 90 and self.patient["oxygen_saturation"] >= 90:
                self.patient["diagnosis"] = 2

            if self.patient["is_infected"] and self.patient["lactate"] >= 6:
                self.patient["diagnosis"] = 4
        
        elif self.patient["diagnosis"] == 4: # Septic Shock
            # Update Vitals
            self.patient["systolic_bp"] -= random.randint(0, 10)
            self.patient["diastolic_bp"] -= random.randint(0, 10)
            self.patient["lactate"] += random.uniform(0, 1.5)
            self.patient["temperature"] += random.uniform(0, .5)
            self.patient["respiratory_rate"] += random.randint(0, 3)
            self.patient["oxygen_saturation"] -= random.randint(1, 5)

            # Update Diagnosis
            if self.patient["lactate"] < 6 and self.patient["systolic_bp"] > 80:
                self.patient["diagnosis"] = 3
            
        # Increment time
        self.time_elapsed += 1

        ### Check if patient is alive ###
        if self.patient["systolic_bp"] < 70 or self.patient["systolic_bp"] > 180 or self.patient["oxygen_saturation"] < 70 or \
                self.patient["respiratory_rate"] > 25 or self.patient["lactate"] >= 8 or self.patient["temperature"] > 42 or \
                self.patient["temperature"] < 35:
            self.is_alive = False

        # Past 12 hours mortality rate of Severe Sepsis and Septic Shock is Very Low for Simulation we assume Death
        elif self.patient["diagnosis"] >= 3 and self.time_elapsed >= 12:
            self.is_alive = False

        # Determine reward
        if self.is_alive and self.patient["diagnosis"] == 0:
            reward = 10
        elif not self.is_alive:
            reward = -10
        else:
            reward = -1

        # Determine if episode is done
        done = not self.is_alive or self.patient["diagnosis"] == 0 or self.time_elapsed >= 12

        return self.get_state(), reward, done


    def get_state(self):
        # Return patient state as a tensor (tensorflow formatted matrix)
        state = [
            self.patient["age"],
            1 if self.patient["sex"] == "male" else 0,
            self.patient["temperature"],
            self.patient["heart_rate"],
            self.patient["respiratory_rate"],
            self.patient["systolic_bp"],
            self.patient["diastolic_bp"],
            self.patient["lactate"],
            self.patient["oxygen_saturation"],
            1 if self.patient["is_infected"] else 0,
            self.patient["diagnosis"],
            self.time_elapsed
        ]
        return tf.reshape(tf.convert_to_tensor(state), (1,12))

## **Deep Q-Learning Network**

In [3]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=10_000)
        self.gamma = 0.99    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()         # Main Model
        self.target_model = self._build_model()  # Target Model
        self.update_target_model()

    def _build_model(self):
        # Neural Network for Deep-Q learning Model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(512, input_dim=self.state_size, activation='relu'))
        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        # Copy the weights from the main model to the target model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        # Adds info to memory deque
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Epsilon Greedy Algorithm
        if np.random.rand() <= self.epsilon: 
            return random.randrange(self.action_size)
        act_values = self.model(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        # Replay Algorithm for learning from batch_size number of past actions 
        minibatch = random.sample(self.memory, batch_size)

        # Initialize empty arrays to store the training data
        states = np.zeros((batch_size, self.state_size))
        targets = np.zeros((batch_size, self.action_size))

        # Fill in the training data
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            states[i] = state
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.target_model(next_state)[0]))
            targets[i] = self.model(state).numpy()
            targets[i][action] = target

        # Use GradientTape to calculate the gradients and optimize the model parameters
        with tf.GradientTape() as tape:
            q_values = self.model(states)
            loss = tf.keras.losses.mean_squared_error(targets, q_values)
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.model.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        if self.epsilon > self.epsilon_min:
            # Decrease epsilon as trainig goes on
            self.epsilon *= self.epsilon_decay

        if len(self.memory) % 50 == 0:
            # Every 50 Steps Set target model's weights to current model weights
            self.update_target_model()

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

### **Training**

In [4]:
# initialize the environment
env = SepsisSimulation()
state_size = env.get_state().shape[1]  # 12 Inputs
action_size = 5                        # 5 Possible Outputs
batch_size = 32                        # Each Step Replay 32 Previous States

# Initialize Tracking Parameters
succ = 0
step_list = []

# initialize the deep Q-learning model
dqn_agent = DQN(state_size, action_size)

# run the game for a specified number of episodes
EPISODES = 1001
ROUND = 100

# Begin Trainning DQN
for e in range(EPISODES):
    # The environment generates a new patient and Gets the Beginning State
    env.new_patient()
    state = env.get_state()
    done = False
    # This loop runs until the episode is done
    while not done:
        action = dqn_agent.act(state)
        next_state, reward, done = env.step(action)
        dqn_agent.remember(state, action, reward, next_state, done)
        state = next_state
        if len(dqn_agent.memory) > batch_size: # If the memory > batch size, the agent learns from it
            dqn_agent.replay(batch_size)

    # Metrics For Tracking During Training
    if reward > 0:
        succ += 1
    step_list.append(env.time_elapsed)

    if e % ROUND == 0 and e != 0:
        print("Episode: {}, % Stablized: {}, Avg Time Taken: {}".format(e, succ/ROUND, sum(step_list)/ROUND))
        succ = 0
        step_list = []

Episode: 100, % Stablized: 0.53, Avg Time Taken: 8.38
Episode: 200, % Stablized: 0.56, Avg Time Taken: 8.28
Episode: 300, % Stablized: 0.78, Avg Time Taken: 7.56
Episode: 400, % Stablized: 0.95, Avg Time Taken: 5.78
Episode: 500, % Stablized: 0.88, Avg Time Taken: 6.67
Episode: 600, % Stablized: 0.93, Avg Time Taken: 6.35
Episode: 700, % Stablized: 0.93, Avg Time Taken: 5.74
Episode: 800, % Stablized: 0.93, Avg Time Taken: 6.31
Episode: 900, % Stablized: 0.94, Avg Time Taken: 6.19
Episode: 1000, % Stablized: 0.95, Avg Time Taken: 5.97


## **Playing Through the Simulation**

In [28]:
import time
from IPython.display import clear_output

# Dictionaries for Visualizing within play_sim
action_list = {0: 'Antipyretic', 1: 'Antibiotics', 2: 'Fluids', 3: 'Vasopressors', 4: 'Oxygen Therapy'}
outcome_list = {10: 'Stablized', -10: 'Death', -1: 'Unknown'}

# For Visualizing the game states as our agent plays
def play_sim(a, e, sleep_time=1, max_steps=12):
    '''
    @param a: Agent
    @param e: Environment
    @param sleep_time: Amount of time between Actions
    @param max_steps: Number of Actions Agent can make
    '''
    done = False
    while not done:
        clear_output(wait=True)
        st = e.get_state()
        q_values = a.model(st).numpy()
        action = np.argmax(q_values)
        print("----------------------------------------------------")
        print("Diagnosis:", e.patient['diagnosis'])
        print("Recommended Treatment:", action_list[action])
        next_state, reward, done = e.step(action)
        print("Updated Patient Info:\n")
        for k, v in e.patient.items():
            print(f"{k.capitalize()}: {v}")
        print("----------------------------------------------------")
        print(f'Time Elapsed: {e.time_elapsed}')
        time.sleep(sleep_time)
        
    print(f"Patient Outcome: {outcome_list[reward]}")

In [5]:
# Generates Testing Environment and Loads in Pretrained Model
test_env = SepsisSimulation()
agent = DQN(12,5)
agent.load('doc_perf.h5')

In [89]:
###   RERUN CELL TO SIMULATE NEW PATIENTS   ###

# Generates New Patient
# Change value between 0-4 for what starting diagnosis you want to test
# Or leave blank for random generation
test_env.new_patient(4) 

# Plays through simulation
play_sim(agent, test_env, 1)

----------------------------------------------------
Diagnosis: 1
Recommended Treatment: Antipyretic
Updated Patient Info:

Age: 74
Sex: male
Temperature: 37.5
Heart_rate: 134
Respiratory_rate: 12
Systolic_bp: 135
Diastolic_bp: 50
Lactate: 2.253693042395456
Oxygen_saturation: 100
Is_infected: False
Diagnosis: 0
----------------------------------------------------
Time Elapsed: 12
Patient Outcome: Stablized
